Для создания симулятора энергопотребления необходимо определить математическую модель, которая учитывает два основных компонента: энергию на вычисления ($E_{comp}$) и энергию на перемещение данных ($E_{data}$).

В нейроморфных и специализированных архитектурах (NPU) основной выигрыш достигается за счет минимизации $E_{data}$, так как перемещение данных из внешней памяти (DRAM) в процессор может потреблять в 100–1000 раз больше энергии, чем сама операция умножения.

Общая энергия ($E_{total}$) для задачи умножения двух матриц размером $N \times N$ рассчитывается как:

$$E_{total} = E_{comp} + E_{data}$$

Где:
- Количество операций (FLOPs) для умножения матриц: $2 \cdot N^3$.
- Объем данных: $3 \cdot N^2 \cdot \text{size\_of\_element}$ (две входные матрицы и одна выходная).

| Архитектура | Энергия на операцию (e_op) | Энергия на перенос байта (e_byte) | Особенности |
|------------|---------------------------|-----------------------------------|-------------|
| GPU (H100/A100) | ~1.0 отн. ед. | Высокая (DRAM ↔ Core) | Огромная мощь, но высокая цена за перемещение данных. |
| TPU (v5/v6) | ~0.5 отн. ед. | Средняя (HBM ↔ Systolic Array) | Систолические массивы минимизируют обращения к памяти внутри чипа. |
| NPU (Edge/In-memory) | ~0.1 отн. ед. | Близка к 0 (Data-in-place) | Данные хранятся там же, где вычисляются (Compute-in-Memory). |

In [1]:
class Architecture:
    """Базовый класс для вычислительных архитектур"""
    def __init__(self, e_op, e_byte):
        self.e_op = e_op      # Энергия на одну операцию (пДж)
        self.e_byte = e_byte  # Энергия на перемещение 1 байта (пДж)

    def calculate_energy(self, N):
        """Общий метод расчета для перемножения матриц NxN"""
        ops = 2 * (N**3)
        data_volume = 3 * (N**2) * 4  # 4 байта на float32
        
        e_comp = ops * self.e_op
        e_data = data_volume * self.e_byte
        return e_comp, e_data

class GPU(Architecture):
    def __init__(self):
        super().__init__(e_op=10, e_byte=100)

class TPU(Architecture):
    def __init__(self):
        super().__init__(e_op=5, e_byte=30)

class NPU(Architecture):
    def __init__(self):
        super().__init__(e_op=1, e_byte=2)

def run_simulation(N):
    architectures = [GPU(), TPU(), NPU()]
    
    print(f"Симуляция энергопотребления (Матрица {N}x{N})")
    for arch in architectures:
        comp, data = arch.calculate_energy(N)
        total = comp + data

        name = arch.__class__.__name__
        
        print(f"{name:3}: Вычисления={comp:.2e} пДж, Данные={data:.2e} пДж, Всего={total:.2e} пДж")

if __name__ == "__main__":
    run_simulation(1024)

Симуляция энергопотребления (Матрица 1024x1024)
GPU: Вычисления=2.15e+10 пДж, Данные=1.26e+09 пДж, Всего=2.27e+10 пДж
TPU: Вычисления=1.07e+10 пДж, Данные=3.77e+08 пДж, Всего=1.11e+10 пДж
NPU: Вычисления=2.15e+09 пДж, Данные=2.52e+07 пДж, Всего=2.17e+09 пДж


GPU: Затраты на данные составляют значительную часть бюджета.

TPU уверенно занимает промежуточное положение. Его архитектура (систолические массивы) оптимизирована под умножение матриц лучше, чем у GPU, что позволило сократить расходы на данные почти в 3.3 раза по сравнению с графическим процессором.

NPU показал себя в 10.5 раз эффективнее, чем GPU, и в 5 раз эффективнее, чем TPU. Итоговое потребление: $2.17 \times 10^9$ пДж против $22.7 \times 10^9$ пДж у GPU. Такой результат достигается за счет радикального снижения затрат как на сами вычисления ($e_{op}$), так и на минимизацию перемещения данных.